In [121]:
import pickle

with open('bert_0.5_final_3.pkl', 'rb') as f:
    data = pickle.load(f)
type(data)
# display(data)
print(data['perturbed'].head(50))
print(data['clean'].head(50))

0    there are some incredibly Stuuupidd people on ...
0    there are some incredibly Stuuupidd people on ...
0    there are some incredibly Stuuupidd people on ...
0                           thats just plain Stuuupidd
0                           thats just plain Stuuupidd
0                           thats just plain Stuuupidd
0    typical empty Stuuupidd reply based on hate ov...
0    typical empty Stuuupidd reply based on hate ov...
0    typical empty Stuuupidd reply based on hate ov...
0                i always knew you were a dirtyyyy cop
0                i always knew you were a dirtyyyy cop
0                i always knew you were a dirtyyyy cop
0                 DUMBBBB response off topic as always
0                 DUMBBBB response off topic as always
0                 DUMBBBB response off topic as always
0                i DAAAAMN no one they damn themselves
0                i daaammn no one they damn themselves
0                i DAMNNNN no one they damn themselves
0    wow t

In [124]:

def locate_pert(row):
    # print(row)
    c,p = row['clean'].split(), row['perturbed'].split()
    for i in range(len(c)):
        if c[i] != p[i]:
            return i
    return -1


distinct_data = data[data['toxicity'] > 0.5].drop_duplicates()
distinct_data['location'] = distinct_data.apply(locate_pert,axis=1)
distinct_data = distinct_data.drop(['perturbed'], axis=1).drop_duplicates()
print(distinct_data.shape)



(1393, 45)


In [83]:

perspective_cache = {}

In [105]:
perspective_cache_copy={k[0]:v for k,v in perspective_cache.items()}
perspective_cache = perspective_cache_copy

In [106]:
import time
from googleapiclient import discovery
from serpapi import GoogleSearch
import pandas as pd 
import json

API_KEY = 'XXXXXXX'

client = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=API_KEY,
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

def perspective_acc(data,t=0.7):
    denominator = [0,0,0,0] #acc_num_of_clean_perspective, acc_num_of_pert_perspective, acc_num_of_cor_perspective_languageTool, acc_num_of_cor_perspective_Google
    acc_num_of_lt = 0
    acc_num_of_google = 0
    total = 0
    for idx,row in data.iterrows():
        time.sleep(2)
        total += 1
        print(total)
        sentence_clean = row['Answer.clean']
        sentence_pert = row['Answer.perturbed']
        sentences = [sentence_clean,sentence_pert]
        # print(sentences)


        for i,sentence in enumerate(sentences):
            analyze_request = {
                'comment': { 'text': sentence },
                'requestedAttributes': {'TOXICITY': {}},
                'languages': ["en"],
                'doNotStore':True
            }
            
            response = client.comments().analyze(body=analyze_request).execute()
            score = 0 if response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"] < t else 1
            
            if score:
                denominator[i] += 1
    
    # print(f"Perspective API Performance for Clean Sentence: {round(denominator[0]/total,2)}")
    # print(f"Perspective API Performance for Perturbed Sentence: {round(denominator[1]/total,2)}")
    return f"clean: {round(denominator[0]/total,2)}; perturb: {round(denominator[1]/total,2)}"
    # break

def google_correct(data):
    total = 0
    cnt = 0
    for idx,row in data.iterrows():
        time.sleep(1)
        total += 1
        perturbed = row['Answer.perturbed']
        clean = row['Answer.clean']
        print(perturbed,clean)
        params = {
            "q": perturbed,
            "hl": "en",
            "gl": "us",
            "api_key": "XXXXXXXX"
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        corrected = results.get('search_information', {}).get('spelling_fix',perturbed)
        if corrected == clean:
            cnt += 1
    return f"cnt: {cnt}; perturb: {round(cnt/total,2)}"


def perspective_api(row, t=0.7,col_name='Answer.perturbed'):
    sentence = row[col_name]
    if not sentence in perspective_cache:
        time.sleep(2)
        analyze_request = {
                    'comment': { 'text': sentence },
                    'requestedAttributes': {'TOXICITY': {}},
                    'languages': ["en"],
                    'doNotStore':True
                }
        
        response = client.comments().analyze(body=analyze_request).execute()
        perspective_cache[sentence] = response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"]
    return perspective_cache[sentence]



In [82]:
from serpapi import GoogleSearch

perturbed = "how areeee you"
clean = "how are you"
# print(self.sentence)
params = {
    "q": perturbed,
    "hl": "en",
    "gl": "us",
    "api_key": "xxxxxxx"
}
search = GoogleSearch(params)
results = search.get_dict()
search_information = results.get('search_information', {})
print(search_information.get('spelling_fix', perturbed))

https://serpapi.com/search
how are you


In [10]:
import json
with open('data/hard_pert.json') as json_file:
    hard_pert = json.load(json_file)
hard_pert['dirty']

['dirtyyyy']

In [6]:
import pandas as pd
control = pd.DataFrame(data=None, columns=data.columns)
control = pd.concat([control, pd.DataFrame({'clean': 'you really need to kill yourself', 'perturbed': 'you really need to ki11 yourself', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'you really need to kill yourself', 'perturbed': 'you really need to ki11 yourself', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'and luke stay out son of a bitch', 'perturbed': 'and luke stay out son of a biiiitch', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'dude this guy is so gay', 'perturbed': 'dude this guy is so GAYYYY', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'i am going to hack your dick account', 'perturbed': 'i am going to hack your dicckkk account', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'oh i yeah i feel like an ass now', 'perturbed': 'oh i yeah i feel like an ASSSSS now', 'rating':5}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'they still cant fill a full bag of suck', 'perturbed': 'they still cant fill a full bag of such', 'rating':1}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'thats just plain stupid', 'perturbed': 'thats just plain styled', 'rating':1}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'dumb response off topic as always', 'perturbed': 'dump response off topic as always', 'rating':1}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'damn you beat me to it', 'perturbed': 'dawn you beat me to it', 'rating':1}, index=[0])], ignore_index=True)
control = pd.concat([control, pd.DataFrame({'clean': 'why you a son of a b', 'perturbed': 'why you a soon of a b', 'rating':1}, index=[0])], ignore_index=True)
# print(control.dtypes)
control[['clean', 'perturbed','rating']]

,clean,perturbed,rating
0,you really need to kill yourself,you really need to ki11 yourself,5
1,you really need to kill yourself,you really need to ki11 yourself,5
2,and luke stay out son of a bitch,and luke stay out son of a biiiitch,5
3,dude this guy is so gay,dude this guy is so GAYYYY,5
4,i am going to hack your dick account,i am going to hack your dicckkk account,5
5,oh i yeah i feel like an ass now,oh i yeah i feel like an ASSSSS now,5
6,they still cant fill a full bag of suck,they still cant fill a full bag of such,1
7,thats just plain stupid,thats just plain styled,1
8,dumb response off topic as always,dump response off topic as always,1
9,damn you beat me to it,dawn you beat me to it,1


In [19]:
import numpy as np
batch_size = 11
data = data[data['toxicity'] > 0.5].drop_duplicates()
nparr = np.array_split(data, batch_size)
for i in range(batch_size):
    # cur = pd.concat([nparr[i], control], ignore_index=True)
    cur = nparr[i]
    cur.to_csv(f"data/mturk_batch/batch_{i}_raw.csv", index=False)
    print(cur.shape)

(204, 45)
(204, 45)
(204, 45)
(204, 45)
(204, 45)
(203, 45)
(203, 45)
(203, 45)
(203, 45)
(203, 45)
(203, 45)


In [8]:
google_cache = {}
bing_cache = {}

In [119]:
import json
    
# Data to be written
    
with open("data/google_cache.json", "w") as outfile:
    json.dump(list(google_cache.items()), outfile)
with open("data/bing_cache.json", "w") as outfile:
    json.dump(list(bing_cache.items()), outfile)
with open("data/perspective_cache.json", "w") as outfile:
    json.dump(list(perspective_cache.items()), outfile)


# load
# with open("file.json","r") as f:
#     your_dict = dict([tuple((tuple(x[0]), x[1])) for x in json.loads(f.read())])

In [120]:
import pandas as pd
import Levenshtein
import requests


def locate_pert(row):
    # print(row)
    c,p = row['Answer.clean'].split(), row['Answer.perturbed'].split()
    for i in range(len(c)):
        if c[i] != p[i]:
            return i
    return -1


def distance(row):
    l = row['location']
    c,p = row['Answer.clean'].split()[l], row['Answer.perturbed'].split()[l]
    return Levenshtein.distance(c.lower(),p.lower())


def show_pert(row):
    l = row['location']
    return row['Answer.perturbed'].split()[l]


def google_api(perturbed_sentence, clean_sentence, l):
    if (perturbed_sentence, clean_sentence, l) in google_cache:
        return google_cache[(perturbed_sentence, clean_sentence, l)]
    params = {
        "q": perturbed_sentence,
        "hl": "en",
        "gl": "us",
        "api_key": "xxxxxx"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    corrected = results.get('search_information', {}).get('spelling_fix',perturbed_sentence)
    if len(corrected.split()) > l and corrected.split()[l].lower() == clean_sentence.split()[l].lower():
        google_cache[(perturbed_sentence, clean_sentence, l)] = (corrected.split()[l].lower(),1)
    else:
        google_cache[(perturbed_sentence, clean_sentence, l)] = (corrected.split()[l].lower(),0) if len(corrected.split()) > l else (corrected,0)
    return google_cache[(perturbed_sentence, clean_sentence, l)]
        


def bing_api(perturbed_sentence, clean_sentence, l):
    if (perturbed_sentence, clean_sentence, l) in bing_cache:
        return bing_cache[(perturbed_sentence, clean_sentence, l)]
    corrected = perturbed_sentence
    api_key = "xxxxx"
    endpoint = "https://api.bing.microsoft.com/v7.0/spellcheck"
    data = {'text': perturbed_sentence}
    params = {
    'mkt':'en-us',
    'mode':'proof'
    }
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Ocp-Apim-Subscription-Key': api_key,
    }
    response = requests.post(endpoint, headers=headers, params=params, data=data)
    for e in response.json().get('flaggedTokens', []):
        if 'token' in e and 'suggestions' in e and e['suggestions']:
            corrected = corrected.replace(e['token'], e['suggestions'][0]['suggestion'])
    if len(corrected.split()) > l and corrected.split()[l].lower() == clean_sentence.split()[l].lower():
        # print(corrected.split()[l].lower(),1)
        bing_cache[(perturbed_sentence, clean_sentence, l)] = (corrected.split()[l].lower(),1)
    else:
        bing_cache[(perturbed_sentence, clean_sentence, l)] = (corrected.split()[l].lower(),0) if len(corrected.split()) > l else (corrected,0)
    return bing_cache[(perturbed_sentence, clean_sentence, l)]


def api_correct(row,func=google_api):
    perturbed_sentence, clean_sentence, l = row['Answer.perturbed'], row['Answer.clean'], row['location']
    result, correct = func(perturbed_sentence, clean_sentence, l)
    result_sentence_list = clean_sentence.split()
    result_sentence_list[l] = result
    return pd.Series([result, ' '.join(result_sentence_list), correct])


df_batch_0 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4978329_batch_results.csv')
print(df_batch_0.shape)
df_batch_1 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4979170_batch_results.csv')
print(df_batch_1.shape)
df_batch_2 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4979419_batch_results.csv')
print(df_batch_2.shape)
df_batch_3 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4981951_batch_results.csv')
print(df_batch_3.shape)
df_batch_4 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4979863_batch_results.csv')
print(df_batch_4.shape)
df_batch_5 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4979911_batch_results.csv')
print(df_batch_5.shape)
df_batch_6 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4980383_batch_results.csv')
print(df_batch_6.shape)
df_batch_7 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4980585_batch_results.csv')
print(df_batch_7.shape)
df_batch_8 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4981021_batch_results.csv')
print(df_batch_8.shape)
df_batch_9 = pd.read_csv('data/mturk_batch/mturk_result/Batch_4981364_batch_results.csv')
print(df_batch_9.shape)

# df = pd.concat([df_batch_0, df_batch_1, df_batch_2, df_batch_3, df_batch_4, df_batch_5, df_batch_6, df_batch_7], ignore_index=True)

df = pd.concat([df_batch_0, df_batch_1, df_batch_2, df_batch_3, df_batch_4, df_batch_5, df_batch_6, df_batch_7, df_batch_8, df_batch_9], ignore_index=True)
print(df.shape)

df['quality'] = df.apply(lambda row: 1 if row['Answer.Very unlikely.1'] else 2 if row['Answer.Unlikely.2'] else 3 if row['Answer.Fair.3'] else 4 if row['Answer.Likely.4'] else 5, axis=1)
df['location'] = df.apply(locate_pert,axis=1)
df['distance'] = df.apply(distance,axis=1)
df['pert_word'] = df.apply(show_pert,axis=1)
print(df.shape)
df = df[df["Answer.pert_location"] == df['location']]
df = df[df["Input.toxicity"] > 0.5]

print(df.shape)

tagged_df = df[['Answer.clean','Answer.perturbed','quality','Answer.time_spent','location', 'distance', 'pert_word']]

grouped_multiple = tagged_df.groupby(['Answer.clean', 'Answer.perturbed', 'location', 'distance', 'pert_word']).agg({'quality': ['mean', 'min', 'max', 'count']})
grouped_multiple.columns = ['quality_mean', 'quality_min', 'quality_max', 'quality_cnt']
grouped_multiple = grouped_multiple.reset_index()
print(grouped_multiple.shape)
grouped_multiple = grouped_multiple[grouped_multiple["quality_cnt"] >= 3]
grouped_multiple = grouped_multiple[grouped_multiple["quality_mean"] > 3]
print(grouped_multiple.shape)

grouped_multiple[['google_correct_word','google_correct_sentence','google_correct']] = grouped_multiple.apply(api_correct, args=(google_api,), axis=1)
grouped_multiple[['bing_correct_word','bing_correct_sentence','bing_correct']] = grouped_multiple.apply(api_correct, args=(bing_api,), axis=1)


grouped_multiple

# df.columns


(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(1060, 84)
(10600, 84)
(10600, 88)
(7701, 88)
(1927, 9)
(1329, 9)


,Answer.clean,Answer.perturbed,location,distance,pert_word,quality_mean,quality_min,quality_max,quality_cnt,google_correct_word,google_correct_sentence,google_correct,bing_correct_word,bing_correct_sentence,bing_correct
0,a bullet to the head dude,a bULLeT to the head dude,1,0,bULLeT,3.800000,3,4,5,bullet,a bullet to the head dude,1,bullet,a bullet to the head dude,1
1,a controversial rat,a controversial Ratt,2,1,Ratt,3.333333,3,4,3,ratt,a controversial ratt,0,ratt,a controversial ratt,0
2,a fast meaningless death is exactly what the e...,a fast meaningless deth is exactly what the ev...,3,1,deth,4.000000,3,5,3,death,a fast meaningless death is exactly what the e...,1,death,a fast meaningless death is exactly what the e...,1
3,a fool and his money,a fol and his money,1,1,fol,3.600000,2,5,5,fool,a fool and his money,1,fool,a fool and his money,1
4,a fool and our money are soon to be parted,a foool and our money are soon to be parted,1,1,foool,3.500000,2,5,4,fool,a fool and our money are soon to be parted,1,fool,a fool and our money are soon to be parted,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,your racist stereotyping is disgusting,your racist stereotyping is DisguSTING,4,0,DisguSTING,4.250000,4,5,4,disgusting,your racist stereotyping is disgusting,1,disgusting,your racist stereotyping is disgusting,1
1923,your talking out of your behind,your talking out of your bEhInD,5,0,bEhInD,4.400000,4,5,5,behind,your talking out of your behind,1,behind,your talking out of your behind,1
1924,your vote was extremely based on stupidity,your vote was extremely based on stupdity,6,1,stupdity,4.000000,3,5,5,stupidity,your vote was extremely based on stupidity,1,stupidity,your vote was extremely based on stupidity,1
1925,youve got a gun time to get shot stop resisting,youve got a gun time to get SHooT stop resisting,7,1,SHooT,3.800000,2,5,5,shot,youve got a gun time to get shot stop resisting,1,shoot,youve got a gun time to get shoot stop resisting,0


In [122]:
grouped_multiple['perspective_result_on_pert'] = grouped_multiple.apply(perspective_api, args=(0.5,'Answer.perturbed'), axis=1)
grouped_multiple['perspective_result_on_clean'] = grouped_multiple.apply(perspective_api, args=(0.5,'Answer.clean'), axis=1)
grouped_multiple['perspective_result_on_bing_correct'] = grouped_multiple.apply(perspective_api, args=(0.5,'bing_correct_sentence'), axis=1)
grouped_multiple['perspective_result_on_google_correct'] = grouped_multiple.apply(perspective_api, args=(0.5,'google_correct_sentence'), axis=1)
# grouped_multiple['perspective_result_on_spell_correct'] = grouped_multiple.apply(perspective_api, args=(0.5,'spell_correct_sentence'), axis=1)
# grouped_multiple['perspective_result_on_neu_correct'] = grouped_multiple.apply(perspective_api, args=(0.5,'neu_correct_sentence'), axis=1)
print("perspective_result_on_pert acc: ", grouped_multiple[grouped_multiple["perspective_result_on_pert"] > 0.5].shape[0] / grouped_multiple.shape[0])
print("perspective_result_on_clean acc: ", grouped_multiple[grouped_multiple["perspective_result_on_clean"] > 0.5].shape[0] / grouped_multiple.shape[0])
print("perspective_result_on_bing_correct acc: ", grouped_multiple[grouped_multiple["perspective_result_on_bing_correct"] > 0.5].shape[0] / grouped_multiple.shape[0])
print("perspective_result_on_google_correct acc: ", grouped_multiple[grouped_multiple["perspective_result_on_google_correct"] > 0.5].shape[0] / grouped_multiple.shape[0])
# print("perspective_result_on_spell_correct acc: ", grouped_multiple["perspective_result_on_spell_correct"].mean())
# print("perspective_result_on_neu_correct acc: ", grouped_multiple["perspective_result_on_neu_correct"].mean())

perspective_result_on_pert acc:  0.671933784800602
perspective_result_on_clean acc:  0.8141459744168548
perspective_result_on_bing_correct acc:  0.7223476297968398
perspective_result_on_google_correct acc:  0.7268623024830699


In [232]:
t_generator = (round(x * 0.05,2) for x in range(0, 20))
for t in t_generator:
    print(t,round(grouped_multiple[grouped_multiple["perspective_result_on_pert"] > t].shape[0] / grouped_multiple.shape[0],3),round(grouped_multiple[grouped_multiple["perspective_result_on_clean"] > t].shape[0] / grouped_multiple.shape[0],3))

0.0 1.0 1.0
0.05 0.998 1.0
0.1 0.994 1.0
0.15 0.987 1.0
0.2 0.983 1.0
0.25 0.977 1.0
0.3 0.956 0.996
0.35 0.924 0.99
0.4 0.875 0.965
0.45 0.792 0.916
0.5 0.672 0.814
0.55 0.547 0.681
0.6 0.389 0.525
0.65 0.254 0.363
0.7 0.175 0.26
0.75 0.126 0.204
0.8 0.07 0.114
0.85 0.04 0.072
0.9 0.01 0.019
0.95 0.0 0.0


In [57]:
grouped_multiple = grouped_multiple[~grouped_multiple['pert_word'].isin(['STUPIDDDDD','stupiddddd','Stuuupidd','stpd'])]

In [64]:
print("google acc: ", grouped_multiple["google_correct"].mean())
print("bing acc: ", grouped_multiple["bing_correct"].mean())
print("spell checker acc: ", grouped_multiple["spell_correct"].mean())
print("neu acc: ", grouped_multiple["neu_correct"].mean())

google acc:  0.7550173010380623
bing acc:  0.6325259515570935
spell checker acc:  0.7280276816608997
neu acc:  0.5826989619377163


In [56]:
grouped_multiple.to_csv('grouped_multiple.csv',index=False)


In [50]:
distance_vs_acc_df = grouped_multiple.groupby(['distance']).agg({'google_correct': ['mean', 'count'],'bing_correct': ['mean', 'count']})
distance_vs_acc_df

google_correct       bing_correct      
                   mean count         mean count
distance                                        
1              0.759259    54     0.240741    54
2              0.418182    55     0.127273    55
3              0.818841   138     0.050725   138
4              0.831731   208     0.019231   208
5              0.300000    30     0.066667    30

In [65]:
pert_word_vs_acc_df = grouped_multiple.groupby(['pert_word']).agg({'google_correct': ['mean', 'count'],'bing_correct': ['mean', 'count']})
pert_word_vs_acc_df.columns = ['google_mean', 'google_cnt', 'bing_mean','bing_cnt']
pert_word_vs_acc_df = pert_word_vs_acc_df.reset_index()
pert_word_vs_acc_df = pert_word_vs_acc_df.sort_values(by='google_mean', ascending=False)
print(pert_word_vs_acc_df.to_string())

      pert_word  google_mean  google_cnt  bing_mean  bing_cnt
63   TRRAAASSHH     1.000000           2   0.500000         2
59     Shooooot     1.000000           1   1.000000         1
80   crazyyyyyy     1.000000           1   0.000000         1
78         blCK     1.000000           2   1.000000         2
77   bitcccccch     1.000000           1   0.000000         1
76    biiitchhh     1.000000           1   0.000000         1
73        baddd     1.000000           1   1.000000         1
72      baaaaad     1.000000           1   0.000000         1
65      UGGGGLY     1.000000           1   0.000000         1
64    Trashhhhh     1.000000           1   0.000000         1
58    Shoooooot     1.000000           1   0.000000         1
82   crrrrazzyy     1.000000           1   1.000000         1
53        PIGGG     1.000000           1   0.000000         1
52         Nutt     1.000000           1   0.000000         1
51         NUTT     1.000000           1   0.000000         1
50      

In [4]:
import requests
import time
from serpapi import GoogleSearch

def google_correct(data):
    total = 0
    cnt = 0
    for idx,row in data.iterrows():
        time.sleep(1)
        total += 1
        perturbed = row['Answer.perturbed']
        clean = row['Answer.clean']
        params = {
            "q": perturbed,
            "hl": "en",
            "gl": "us",
            "api_key": "xxxx"
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        corrected = results.get('search_information', {}).get('spelling_fix',perturbed)
        # print(corrected, corrected.lower() == clean.lower())
        if len(corrected.split()) > int(row['location']) and corrected.split()[int(row['location'])].lower() == clean.split()[int(row['location'])].lower():
            cnt += 1
        else:
            print(f"pert: {perturbed}, clean: {clean}, corrected: {corrected}")
    return f"cnt: {cnt}; total: {total}; perturb: {round(cnt/total,2)}"



def bing_correct(data):
    total = 0
    cnt = 0
    for idx,row in data.iterrows():
        time.sleep(1)
        total += 1
        perturbed = row['Answer.perturbed']
        clean = row['Answer.clean']
        corrected = perturbed

        api_key = "xxxxxxx"
        endpoint = "https://api.bing.microsoft.com/v7.0/spellcheck"
        data = {'text': perturbed}

        params = {
        'mkt':'en-us',
        'mode':'proof'
        }
        headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Ocp-Apim-Subscription-Key': api_key,
        }
        response = requests.post(endpoint, headers=headers, params=params, data=data)
        # print(response.json())
        for e in response.json().get('flaggedTokens', []):
            if 'token' in e and 'suggestions' in e and e['suggestions']:
                corrected = corrected.replace(e['token'], e['suggestions'][0]['suggestion'])
        # print(corrected)
        if len(corrected.split()) > int(row['location']) and corrected.split()[int(row['location'])].lower() == clean.split()[int(row['location'])].lower():
            cnt += 1
        else:
            pass
            # print(f"pert: {perturbed}, clean: {clean}, corrected: {corrected}")
    return f"cnt: {cnt}; total: {total}; perturb: {round(cnt/total,2)}"



# print(grouped_multiple.columns)
print(google_correct(grouped_multiple[grouped_multiple["quality_mean"] > 3]))
print(bing_correct(grouped_multiple[grouped_multiple["quality_mean"] > 3]))

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
pert: another stpd comment, clean: another stupid comment, corrected: another stpd comment
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
pert: astonishingly DUMBBBB false equivalence here did you finish high school, clean: astonishingly dumb false equivalence here did you finish high school, corrected: astonishingly DUMBBBB false equivalence here did you finish high school
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
pert: because he and they are stpd when it comes 

In [110]:
grouped_multiple[grouped_multiple["quality_mean"] > 3].iloc[125]

Answer.clean           what truth pizza gate you cant fix stupid
Answer.perturbed    what truth pizza gate you cant fix Stuuupidd
location                                                       7
quality_mean                                                 3.6
quality_min                                                    3
quality_max                                                    4
Name: 156, dtype: object

In [111]:

params = {
    "q": 'what truth pizza gate you cant fix Stuuupidd',
    "hl": "en",
    "gl": "us",
    "api_key": "xxxxxx"
}
search = GoogleSearch(params)
results = search.get_dict()
corrected = results.get('search_information', {}).get('spelling_fix',perturbed)
print(corrected)

https://serpapi.com/search
what truth pizzagate you can't fix Stupid


In [19]:
spam_df = df[df["Answer.pert_location"] != df['location']]
spam_df
# spam_df.groupby(['Answer.clean', 'Answer.perturbed']).filter(lambda x: len(x) > 2)

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.clean,Answer.pert_location,Answer.perturbed,Answer.time_spent,Approve,Reject,quality,location,distance,pert_word


In [79]:
print(data.shape)
test_data = data.groupby(['clean','perturbed']).agg({'toxicity': ['mean', 'min', 'max']})
test_data[test_data['toxicity']["min"] != test_data['toxicity']['max']]

(2238, 45)


toxicity  \
                                                                mean   
clean                       perturbed                                  
are you really that stupid  are you really that Stuuupidd   0.903941   
are you that stupid         are you that Stuuupidd          0.930238   
because you are that stupid because you are that Stuuupidd  0.955592   
bring it fat boy            bring it FATT boy               0.562083   
                            bring it Fatt boy               0.562083   
...                                                              ...   
you make me sick            you make me Siiiick             0.688390   
                            you make me sicckkk             0.688390   
                            you make me sickkkk             0.688390   
you suck bill walker        you SUUCK bill walker           0.883333   
                            you suckkkk bill walker         0.883333   

                                                                                
                                                                 min       max  
clean                       perturbed                                           
are you really that stupid  are you really that Stuuupidd   0.879310  0.928571  
are you that stupid         are you that Stuuupidd          0.886792  0.973684  
because you are that stupid because you are that Stuuupidd  0.937500  0.973684  
bring it fat boy            bring it FATT boy               0.537500  0.586667  
                            bring it Fatt boy               0.537500  0.586667  
...                                                              ...       ...  
you make me sick            you make me Siiiick             0.673077  0.703704  
                            you make me sicckkk             0.673077  0.703704  
                            you make me sickkkk             0.673077  0.703704  
you suck bill walker        you SUUCK bill walker           0.833333  0.933333  
                            you suckkkk bill walker         0.833333  0.933333  

[61 rows x 3 columns]

In [56]:
# are you really that stupid

with open('bert_0.5_final_3.pkl', 'rb') as f:
    raw_data = pickle.load(f)

raw_data[raw_data["clean"] == "are you really that stupid"]

,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,...,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count,clean,perturbed
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd
0,2017-01-22 17:57:45.880275+00,54.0,878033.0,163340.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.0,are you really that stupid,are you really that Stuuupidd
0,2017-01-22 17:57:45.880275+00,54.0,878033.0,163340.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.0,are you really that stupid,are you really that Stuuupidd
0,2017-01-22 17:57:45.880275+00,54.0,878033.0,163340.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.0,are you really that stupid,are you really that Stuuupidd


In [59]:
jigsaw_df = pd.read_csv("data/jigsaw.csv")

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
592955,7060854,are you really that stupid?,test,2017-06-22 01:18:32.437596+00,102,5455578.0,347067,rejected,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42


In [78]:
# jigsaw_df[jigsaw_df["comment_text"] == "are you really that stupid?"]
# jigsaw_df[jigsaw_df["created_date"] == "2017-01-22 17:57:45.880275+00"][['created_date','comment_text','toxicity']]
jigsaw_df[jigsaw_df['comment_text'].isin(['are you really that stupid?','Are you really that stupid!?'])][['created_date','comment_text','toxicity']]

,created_date,comment_text,toxicity
592955,2017-06-22 01:18:32.437596+00,are you really that stupid?,0.928571
597795,2017-01-22 17:57:45.880275+00,Are you really that stupid!?,0.879310


In [58]:
raw_data[raw_data["created_date"] == "2017-06-22 01:18:32.437596+00"]

,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,...,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count,clean,perturbed
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd
0,2017-06-22 01:18:32.437596+00,102.0,5455578.0,347067.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,42.0,are you really that stupid,are you really that Stuuupidd


In [154]:
import json
import requests
API_URL = "https://api-inference.huggingface.co/models/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two"
API_TOKEN = "xxxxxx"
headers = {"Authorization": f"Bearer {API_TOKEN}"}
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))
data = query("indeed what FOL would embrace such a religion")
data[0][1]

{'label': 'ABUSIVE', 'score': 0.4526337683200836}

In [135]:
bert_cache = {}

In [184]:

roberta_cache = {}

In [189]:
twitter_roberta_cache = {}

In [204]:
huggingface_cache = {}
for k,v in bert_cache.items():
    huggingface_cache[('Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two',k)] = v
for k,v in roberta_cache.items():
    huggingface_cache[('facebook/roberta-hate-speech-dynabench-r4-target',k)] = v
for k,v in twitter_roberta_cache.items():
    huggingface_cache[('cardiffnlp/twitter-roberta-base-hate',k)] = v

In [211]:
def bert_predict(row,col_name='Answer.perturbed'):
    sentence = row[col_name]
    if sentence not in bert_cache:
        time.sleep(2)
        API_URL = "https://api-inference.huggingface.co/models/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two"
        API_TOKEN = "xxxxx"
        headers = {"Authorization": f"Bearer {API_TOKEN}"}
        data = json.dumps(sentence)
        response = requests.request("POST", API_URL, headers=headers, data=data)
        res = json.loads(response.content.decode("utf-8"))
        print(res)
        while 'error' in res or len(res[0]) < 2 or res[0][1]['label'] not in ['ABUSIVE', 'NORMAL']:
            print(sentence, res)
            time.sleep(30)
            response = requests.request("POST", API_URL, headers=headers, data=data)
            res = json.loads(response.content.decode("utf-8"))
        bert_cache[sentence] = res[0][0]['score'] if res[0][0]['label'] == 'ABUSIVE' else res[0][1]['score'] 
    return bert_cache[sentence]


def roberta_predict(row,col_name='Answer.perturbed', roberta_cache=twitter_roberta_cache):
    sentence = row[col_name]
    if sentence not in roberta_cache:
        time.sleep(2)
        API_URL = "https://api-inference.huggingface.co/models/cardiffnlp/twitter-roberta-base-hate"
        # API_URL = "https://api-inference.huggingface.co/models/facebook/roberta-hate-speech-dynabench-r4-target"
        API_TOKEN = "xxxxxx"
        headers = {"Authorization": f"Bearer {API_TOKEN}"}
        data = json.dumps(sentence)
        response = requests.request("POST", API_URL, headers=headers, data=data)
        res = json.loads(response.content.decode("utf-8"))
        print(res)
        while 'error' in res or len(res[0]) < 2 or res[0][1]['label'] not in ['hate', 'nothate', 'non-hate']:
            print(sentence, res)
            time.sleep(30)
            response = requests.request("POST", API_URL, headers=headers, data=data)
            res = json.loads(response.content.decode("utf-8"))
        roberta_cache[sentence] = res[0][0]['score'] if res[0][0]['label'] == 'hate' else res[0][1]['score'] 
    return roberta_cache[sentence]



def huggingface_predict(row,col_name, model_name, label_set, positive_label):
    sentence = row[col_name]
    if (model_name,sentence) not in huggingface_cache:
        time.sleep(2)
        API_URL = "https://api-inference.huggingface.co/models/" + model_name
        API_TOKEN = "xxxxx"
        headers = {"Authorization": f"Bearer {API_TOKEN}"}
        data = json.dumps(sentence)
        response = requests.request("POST", API_URL, headers=headers, data=data)
        res = json.loads(response.content.decode("utf-8"))
        print(res)
        while 'error' in res or len(res[0]) < 2 or res[0][1]['label'] not in label_set:
            print(sentence, res)
            time.sleep(30)
            response = requests.request("POST", API_URL, headers=headers, data=data)
            res = json.loads(response.content.decode("utf-8"))
        huggingface_cache[(model_name,sentence)] = res[0][0]['score'] if res[0][0]['label'] == positive_label else res[0][1]['score'] 
    return huggingface_cache[(model_name,sentence)]

In [194]:
grouped_multiple['bert_result_on_clean'] = grouped_multiple.apply(bert_predict, args=('Answer.clean',), axis=1)
print("bert_result_on_clean acc: ", grouped_multiple[grouped_multiple["bert_result_on_clean"] > 0.5].shape[0] / grouped_multiple.shape[0])
# grouped_multiple['perspective_result_on_pert'] = grouped_multiple.apply(perspective_api, args=('Answer.perturbed'), axis=1)
grouped_multiple['bert_result_on_perturb'] = grouped_multiple.apply(bert_predict, args=('Answer.perturbed',), axis=1)
print("bert_result_on_perturb acc: ", grouped_multiple[grouped_multiple["bert_result_on_perturb"] > 0.5].shape[0] / grouped_multiple.shape[0])



bert_result_on_clean acc:  0.5673438675696012
bert_result_on_perturb acc:  0.5033860045146726


In [203]:
grouped_multiple['roberta_result_on_clean'] = grouped_multiple.apply(roberta_predict, args=('Answer.clean',), axis=1)
print("roberta_result_on_clean acc: ", grouped_multiple[grouped_multiple["roberta_result_on_clean"] > 0.07].shape[0] / grouped_multiple.shape[0])
grouped_multiple['roberta_result_on_perturb'] = grouped_multiple.apply(roberta_predict, args=('Answer.perturbed',), axis=1)
print("roberta_result_on_perturb acc: ", grouped_multiple[grouped_multiple["roberta_result_on_perturb"] > 0.07].shape[0] / grouped_multiple.shape[0])



roberta_result_on_clean acc:  0.7133182844243793
roberta_result_on_perturb acc:  0.6132430398796087


# RoBERTa

In [219]:
model_name = 'cardiffnlp/roberta-base-offensive'
label_set = set(['offensive', 'non-offensive'])
pos_label = 'offensive'

grouped_multiple[model_name + '_on_clean'] = grouped_multiple.apply(huggingface_predict, args=('Answer.clean',model_name,label_set,pos_label,), axis=1)
print("clean acc: ", grouped_multiple[grouped_multiple[model_name + '_on_clean'] > 0.5].shape[0] / grouped_multiple.shape[0])
grouped_multiple[model_name + '_on_perturb'] = grouped_multiple.apply(huggingface_predict, args=('Answer.perturbed',model_name,label_set,pos_label,), axis=1)
print("perturb acc: ", grouped_multiple[grouped_multiple[model_name + '_on_perturb'] > 0.5].shape[0] / grouped_multiple.shape[0])



clean acc:  0.9149736644093304
perturb acc:  0.5229495861550038


# BERT

In [228]:
model_name = 'cardiffnlp/bertweet-base-offensive'
label_set = set(['LABEL_0', 'LABEL_1'])
pos_label = 'LABEL_1'

grouped_multiple[model_name + '_on_clean'] = grouped_multiple.apply(huggingface_predict, args=('Answer.clean',model_name,label_set,pos_label,), axis=1)
print("clean acc: ", grouped_multiple[grouped_multiple[model_name + '_on_clean'] > 0.5].shape[0] / grouped_multiple.shape[0])
grouped_multiple[model_name + '_on_perturb'] = grouped_multiple.apply(huggingface_predict, args=('Answer.perturbed',model_name,label_set,pos_label,), axis=1)
print("perturb acc: ", grouped_multiple[grouped_multiple[model_name + '_on_perturb'] > 0.5].shape[0] / grouped_multiple.shape[0])



clean acc:  0.8969149736644093
perturb acc:  0.5575620767494357


In [229]:
t_generator = (round(x * 0.01,2) for x in range(0, 101))
for t in t_generator:
    print(t,round(grouped_multiple[grouped_multiple["cardiffnlp/bertweet-base-offensive_on_perturb"] > t].shape[0] / grouped_multiple.shape[0],3),round(grouped_multiple[grouped_multiple["cardiffnlp/bertweet-base-offensive_on_clean"] > t].shape[0] / grouped_multiple.shape[0],3))

0.0 1.0 1.0
0.01 1.0 1.0
0.02 1.0 1.0
0.03 1.0 1.0
0.04 1.0 1.0
0.05 0.999 1.0
0.06 0.997 1.0
0.07 0.995 0.999
0.08 0.986 0.999
0.09 0.981 0.998
0.1 0.968 0.998
0.11 0.956 0.998
0.12 0.944 0.998
0.13 0.93 0.998
0.14 0.919 0.998
0.15 0.91 0.998
0.16 0.898 0.998
0.17 0.888 0.998
0.18 0.877 0.998
0.19 0.862 0.998
0.2 0.857 0.995
0.21 0.848 0.995
0.22 0.845 0.992
0.23 0.832 0.989
0.24 0.822 0.989
0.25 0.808 0.983
0.26 0.802 0.983
0.27 0.795 0.98
0.28 0.783 0.98
0.29 0.772 0.979
0.3 0.758 0.978
0.31 0.748 0.974
0.32 0.74 0.973
0.33 0.733 0.971
0.34 0.722 0.97
0.35 0.709 0.965
0.36 0.703 0.961
0.37 0.69 0.96
0.38 0.684 0.957
0.39 0.673 0.953
0.4 0.667 0.95
0.41 0.657 0.946
0.42 0.649 0.942
0.43 0.643 0.936
0.44 0.631 0.933
0.45 0.618 0.929
0.46 0.605 0.918
0.47 0.594 0.913
0.48 0.582 0.909
0.49 0.57 0.902
0.5 0.558 0.897
0.51 0.546 0.895
0.52 0.536 0.892
0.53 0.522 0.883
0.54 0.513 0.882
0.55 0.501 0.871
0.56 0.486 0.865
0.57 0.472 0.854
0.58 0.46 0.846
0.59 0.445 0.838
0.6 0.437 0.826
0.61 